In [28]:
import os
import pandas as pd
import numpy as np
from numpy import where
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, auc, roc_auc_score, roc_curve, classification_report, cohen_kappa_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance

In [29]:
from sklearn.preprocessing import RobustScaler

In [30]:
#df = pd.read_csv('df_fe_combined.csv')
df = pd.read_csv('Data/Processed_Datasets/Combined_tables/df_fe_combined.csv', index_col=0)

In [31]:
df = df.sort_values(['MONTH', '1000_GRID_ID'])

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65688 entries, 4 to 69256
Data columns (total 72 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   1000_GRID_ID                                                         65688 non-null  object 
 1   MONTH                                                                65688 non-null  object 
 2   CODEMAMROT                                                           65688 non-null  object 
 3   CRIME_COUNT                                                          65688 non-null  float64
 4   Infractions entrainant la mort_CRIME_COUNT                           65688 non-null  float64
 5   Introduction_CRIME_COUNT                                             65688 non-null  float64
 6   Méfait_CRIME_COUNT                                                   65688 non-null  float64
 7   Vol 

In [33]:
df.describe(include='all')

,1000_GRID_ID,MONTH,CODEMAMROT,CRIME_COUNT,Infractions entrainant la mort_CRIME_COUNT,Introduction_CRIME_COUNT,Méfait_CRIME_COUNT,Vol dans / sur véhicule à moteur_CRIME_COUNT,Vol de véhicule à moteur_CRIME_COUNT,Vols qualifiés_CRIME_COUNT,...,UNIQUE_FIRE_STATION_COUNT_4_MONTH_BACKWARD_SHIFT,UNIQUE_DIVISION_COUNT_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_SUM_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_MEAN_4_MONTH_BACKWARD_SHIFT,FIRE_INCIDENT_QUATERLY_MEAN,FIRE_RISK_MONTHLY_RANK,FIRE_RISK_QUATERLY_RANK,FIRE_RISK_RANK_MEAN,FIRE_RISK,FIRE_RISK_1_MONTH_FORWARD_SHIFT
count,65688,65688,65688,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,...,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688,65688
unique,714,92,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5
top,s24eLreYAME=,2021-08,REM33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low Fire-Risk,Low Fire-Risk
freq,92,714,5428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29831,30063
mean,NaN,NaN,NaN,2.922086,0.002664,0.836713,0.638945,0.776869,0.513184,0.153711,...,1.048258,0.797954,23.486877,1.322980,2.041248,20.459399,47.296569,33.877984,NaN,NaN
std,NaN,NaN,NaN,5.410334,0.053003,1.981212,1.527666,1.915722,1.148182,0.552054,...,0.919351,0.622871,37.567125,1.289110,3.387876,3.823499,11.158934,7.195556,NaN,NaN
min,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,43.000000,31.500000,NaN,NaN
50%,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,9.000000,1.394616,0.750000,21.000000,50.000000,35.500000,NaN,NaN
75%,NaN,NaN,NaN,4.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,2.000000,1.000000,32.000000,1.928571,2.750000,23.000000,54.000000,38.500000,NaN,NaN


In [34]:
df.tail()

,1000_GRID_ID,MONTH,CODEMAMROT,CRIME_COUNT,Infractions entrainant la mort_CRIME_COUNT,Introduction_CRIME_COUNT,Méfait_CRIME_COUNT,Vol dans / sur véhicule à moteur_CRIME_COUNT,Vol de véhicule à moteur_CRIME_COUNT,Vols qualifiés_CRIME_COUNT,...,UNIQUE_FIRE_STATION_COUNT_4_MONTH_BACKWARD_SHIFT,UNIQUE_DIVISION_COUNT_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_SUM_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_MEAN_4_MONTH_BACKWARD_SHIFT,FIRE_INCIDENT_QUATERLY_MEAN,FIRE_RISK_MONTHLY_RANK,FIRE_RISK_QUATERLY_RANK,FIRE_RISK_RANK_MEAN,FIRE_RISK,FIRE_RISK_1_MONTH_FORWARD_SHIFT
68868,zZMpExLs4dw=,2022-12,66062,7.0,0.0,2.0,1.0,1.0,3.0,0.0,...,4.0,2.0,58.0,2.636364,7.25,13.0,30.0,21.5,Moderate Fire-Risk,Moderate Fire-Risk
68965,zsOtd-zvzaI=,2022-12,REM20,8.0,0.0,2.0,4.0,1.0,1.0,0.0,...,3.0,1.0,104.0,2.212766,16.00,9.0,9.0,9.0,High Fire-Risk,High Fire-Risk
69062,zsz3DbEvcrg=,2022-12,REM25,4.0,0.0,0.0,1.0,3.0,0.0,0.0,...,2.0,1.0,42.0,1.750000,1.50,19.0,53.0,36.0,Low Fire-Risk,Low Fire-Risk
69159,ztpTb_cD1hM=,2022-12,66007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,10.0,2.500000,0.75,21.0,56.0,38.5,Lowest Fire-Risk,Low Fire-Risk
69256,zzht2gM06A0=,2022-12,REM17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,7.0,3.500000,0.00,21.0,59.0,40.0,Lowest Fire-Risk,Low Fire-Risk


In [35]:
df.columns

Index(['1000_GRID_ID', 'MONTH', 'CODEMAMROT', 'CRIME_COUNT',
       'Infractions entrainant la mort_CRIME_COUNT',
       'Introduction_CRIME_COUNT', 'Méfait_CRIME_COUNT',
       'Vol dans / sur véhicule à moteur_CRIME_COUNT',
       'Vol de véhicule à moteur_CRIME_COUNT', 'Vols qualifiés_CRIME_COUNT',
       'INCIDENT_COUNT', '1-REPOND_INCIDENT_COUNT', 'AUTREFEU_INCIDENT_COUNT',
       'Alarmes-incendies_INCIDENT_COUNT', 'FAU-ALER_INCIDENT_COUNT',
       'INCENDIE_INCIDENT_COUNT', 'NOUVEAU_INCIDENT_COUNT',
       'SANS FEU_INCIDENT_COUNT', 'FIRE_INCIDENT_COUNT',
       'UNIQUE_FIRE_STATION_COUNT', 'UNIQUE_DIVISION_COUNT',
       'UNITS_DEPLOYED_SUM', 'UNITS_DEPLOYED_MEAN',
       'temperature_2m_max (°C)_MEAN', 'temperature_2m_min (°C)_MEAN',
       'temperature_2m_mean (°C)_MEAN', 'shortwave_radiation_sum (MJ/m²)_MEAN',
       'precipitation_sum (mm)_MEAN', 'rain_sum (mm)_MEAN',
       'snowfall_sum (cm)_MEAN', 'windspeed_10m_max (km/h)_MEAN',
       'windgusts_10m_max (km/h)_MEAN', '

## Drop unecessary columns

In [36]:
df.drop(columns=['CODEMAMROT', 'CRIME_COUNT', 'YEAR',
                 'Infractions entrainant la mort_CRIME_COUNT',
                 'Introduction_CRIME_COUNT', 'Méfait_CRIME_COUNT',
                 'Vol dans / sur véhicule à moteur_CRIME_COUNT',
                 'Vol de véhicule à moteur_CRIME_COUNT', 'Vols qualifiés_CRIME_COUNT',
                 'INCIDENT_COUNT', '1-REPOND_INCIDENT_COUNT', 'AUTREFEU_INCIDENT_COUNT',
                 'Alarmes-incendies_INCIDENT_COUNT', 'FAU-ALER_INCIDENT_COUNT',
                 'INCENDIE_INCIDENT_COUNT', 'NOUVEAU_INCIDENT_COUNT',
                 'SANS FEU_INCIDENT_COUNT', 'FIRE_INCIDENT_COUNT',
                 'UNIQUE_FIRE_STATION_COUNT', 'UNIQUE_DIVISION_COUNT',
                 'UNITS_DEPLOYED_SUM', 'UNITS_DEPLOYED_MEAN', 'AIRE',
                 'FIRE_INCIDENT_QUATERLY_MEAN', 'FIRE_RISK_MONTHLY_RANK',
                 'FIRE_RISK_QUATERLY_RANK', 'FIRE_RISK_RANK_MEAN', 'FIRE_RISK'], inplace=True)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65688 entries, 4 to 69256
Data columns (total 44 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   1000_GRID_ID                                                         65688 non-null  object 
 1   MONTH                                                                65688 non-null  object 
 2   temperature_2m_max (°C)_MEAN                                         65688 non-null  float64
 3   temperature_2m_min (°C)_MEAN                                         65688 non-null  float64
 4   temperature_2m_mean (°C)_MEAN                                        65688 non-null  float64
 5   shortwave_radiation_sum (MJ/m²)_MEAN                                 65688 non-null  float64
 6   precipitation_sum (mm)_MEAN                                          65688 non-null  float64
 7   rain

## Label Encoding

In [38]:
le = LabelEncoder()
df['FIRE_RISK_1_MONTH_FORWARD_SHIFT'] = le.fit_transform(
    df['FIRE_RISK_1_MONTH_FORWARD_SHIFT'])

In [39]:
mapping_target_classes = dict(zip(le.classes_, range(len(le.classes_))))

In [40]:
print(mapping_target_classes)

{'High Fire-Risk': 0, 'Highest Fire-Risk': 1, 'Low Fire-Risk': 2, 'Lowest Fire-Risk': 3, 'Moderate Fire-Risk': 4}


In [41]:
df.describe(include='all')

,1000_GRID_ID,MONTH,temperature_2m_max (°C)_MEAN,temperature_2m_min (°C)_MEAN,temperature_2m_mean (°C)_MEAN,shortwave_radiation_sum (MJ/m²)_MEAN,precipitation_sum (mm)_MEAN,rain_sum (mm)_MEAN,snowfall_sum (cm)_MEAN,windspeed_10m_max (km/h)_MEAN,...,FAU-ALER_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,INCENDIE_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,NOUVEAU_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,SANS FEU_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,FIRE_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,UNIQUE_FIRE_STATION_COUNT_4_MONTH_BACKWARD_SHIFT,UNIQUE_DIVISION_COUNT_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_SUM_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_MEAN_4_MONTH_BACKWARD_SHIFT,FIRE_RISK_1_MONTH_FORWARD_SHIFT
count,65688,65688,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,...,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000,65688.000000
unique,714,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,s24eLreYAME=,2021-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,92,714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,11.327581,2.723442,7.071079,13.507968,3.283813,2.661760,0.456079,17.170002,...,0.010870,0.143527,0.000046,2.421081,2.030797,1.048258,0.797954,23.486877,1.322980,2.528955
std,NaN,NaN,11.153119,11.106126,11.017013,6.645252,2.759013,2.697419,0.860901,2.815909,...,0.108005,0.439443,0.006758,4.011847,3.650700,0.919351,0.622871,37.567125,1.289110,0.865706
min,NaN,NaN,-21.400000,-31.400000,-25.400000,0.460000,0.000000,0.000000,0.000000,5.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,0.846711,-6.974194,-2.700000,6.930000,1.733333,1.016667,0.000000,15.335929,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,NaN,NaN,12.166667,4.155159,8.000000,13.666258,2.959175,2.226667,0.000000,17.054839,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,9.000000,1.394616,2.000000
75%,NaN,NaN,21.856899,13.102312,17.426667,19.296667,4.092700,3.587625,0.688333,18.653333,...,0.000000,0.000000,0.000000,3.000000,3.000000,2.000000,1.000000,32.000000,1.928571,3.000000


## Train/Test split

In [42]:
df['MONTH'] = pd.to_datetime(df['MONTH'], format='%Y-%m')

In [43]:
df = df.sort_values(['MONTH', '1000_GRID_ID'])
df.reset_index(drop=True)

,1000_GRID_ID,MONTH,temperature_2m_max (°C)_MEAN,temperature_2m_min (°C)_MEAN,temperature_2m_mean (°C)_MEAN,shortwave_radiation_sum (MJ/m²)_MEAN,precipitation_sum (mm)_MEAN,rain_sum (mm)_MEAN,snowfall_sum (cm)_MEAN,windspeed_10m_max (km/h)_MEAN,...,FAU-ALER_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,INCENDIE_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,NOUVEAU_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,SANS FEU_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,FIRE_INCIDENT_COUNT_4_MONTH_BACKWARD_SHIFT,UNIQUE_FIRE_STATION_COUNT_4_MONTH_BACKWARD_SHIFT,UNIQUE_DIVISION_COUNT_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_SUM_4_MONTH_BACKWARD_SHIFT,UNITS_DEPLOYED_MEAN_4_MONTH_BACKWARD_SHIFT,FIRE_RISK_1_MONTH_FORWARD_SHIFT
0,-1xcNei0iJs=,2015-05-01,21.137931,9.587931,15.363793,20.352759,2.762069,2.762069,0.000000,18.096552,...,0.0,0.0,0.0,11.0,7.0,2.0,2.0,90.0,1.730769,0
1,-5jH5xsoao4=,2015-05-01,19.829630,8.874074,14.203704,21.430000,2.292593,2.292593,0.000000,17.725926,...,0.0,0.0,0.0,3.0,4.0,1.0,1.0,43.0,1.653846,2
2,-HbRtrodjf8=,2015-05-01,21.800000,9.025000,15.400000,23.080000,0.600000,0.600000,0.000000,17.125000,...,0.0,0.0,0.0,2.0,3.0,2.0,1.0,18.0,1.800000,2
3,-VjiKSd8OxM=,2015-05-01,21.727083,9.745833,15.912500,23.007500,1.629167,1.629167,0.000000,18.258333,...,0.0,0.0,0.0,12.0,6.0,2.0,2.0,84.0,1.714286,4
4,-fjD6s-8vdo=,2015-05-01,22.492593,12.040741,16.940741,19.905926,3.525926,3.525926,0.000000,22.537037,...,0.0,0.0,0.0,3.0,0.0,2.0,3.0,26.0,1.529412,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65683,zZMpExLs4dw=,2022-12-01,-0.100000,-6.136585,-3.046341,4.172683,4.834146,2.617073,1.551951,19.717073,...,0.0,1.0,0.0,8.0,2.0,4.0,2.0,58.0,2.636364,4
65684,zsOtd-zvzaI=,2022-12-01,0.020370,-5.520370,-2.779630,5.008519,3.135185,1.966667,0.817963,19.755556,...,0.0,0.0,0.0,11.0,14.0,3.0,1.0,104.0,2.212766,0
65685,zsz3DbEvcrg=,2022-12-01,0.942308,-4.980769,-2.019231,4.634231,3.796154,2.369231,0.998846,19.400000,...,0.0,0.0,0.0,3.0,7.0,2.0,1.0,42.0,1.750000,2
65686,ztpTb_cD1hM=,2022-12-01,0.248387,-5.900000,-2.732258,4.384516,3.961290,2.374194,1.110968,19.032258,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,10.0,2.500000,2


In [44]:
# Use this month as the split between the training and testing sets
split_month = pd.to_datetime('2021-01', format='%Y-%m')
groups = df.groupby('1000_GRID_ID')
train_dfs, test_dfs = [], []

for name, group in groups:
    split_index = (group['MONTH'] == split_month).idxmax()

    train_df = group.loc[group['MONTH'] <
                         split_month.strftime('%Y-%m')]   # Training data
    test_df = group.loc[group['MONTH'] >=
                        split_month.strftime('%Y-%m')]   # Testing data

    train_dfs.append(train_df)
    test_dfs.append(test_df)

# Concatenate the training and testing sets for all grids
train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)

# Split the target variable and feature variables separately for both the training and testing sets
# Features (includes the grid and MONTH columns)
X_train = train_df.drop(
    ['1000_GRID_ID', 'MONTH', 'FIRE_RISK_1_MONTH_FORWARD_SHIFT'], axis=1)
# One-hot encoded target variable
y_train = train_df[['FIRE_RISK_1_MONTH_FORWARD_SHIFT']]
# Features (includes the grid and MONTH columns)
X_test = test_df.drop(
    ['1000_GRID_ID', 'MONTH', 'FIRE_RISK_1_MONTH_FORWARD_SHIFT'], axis=1)
# One-hot encoded target variable
y_test = test_df[['FIRE_RISK_1_MONTH_FORWARD_SHIFT']]

# Verify the shapes of the training and testing sets
print("Training set shape: ", X_train.shape, y_train.shape)
print("Testing set shape: ", X_test.shape, y_test.shape)

Training set shape:  (48552, 41) (48552, 1)
Testing set shape:  (17136, 41) (17136, 1)


In [45]:
X_train.columns

Index(['temperature_2m_max (°C)_MEAN', 'temperature_2m_min (°C)_MEAN',
       'temperature_2m_mean (°C)_MEAN', 'shortwave_radiation_sum (MJ/m²)_MEAN',
       'precipitation_sum (mm)_MEAN', 'rain_sum (mm)_MEAN',
       'snowfall_sum (cm)_MEAN', 'windspeed_10m_max (km/h)_MEAN',
       'windgusts_10m_max (km/h)_MEAN', 'winddirection_10m_dominant (°)_MEAN',
       'et0_fao_evapotranspiration (mm)_MEAN', 'PROPERTY_ASSESSMENT_COUNT',
       'Condominium_PROPERTY_ASSESSMENT_COUNT',
       'Régulier_PROPERTY_ASSESSMENT_COUNT', 'HEIGHT_ABOVE_GROUND_MEAN',
       'DWELLING_MEAN', 'CONSTRUCTION_YEAR_MEAN', 'LAND_AREA_MEAN',
       'BUILDING_AREA_MEAN', 'POPULATION_SUM', 'POPULATION_DENSITY',
       'CRIME_COUNT_4_MONTH_BACKWARD_SHIFT',
       'Infractions entrainant la mort_CRIME_COUNT_4_MONTH_BACKWARD_SHIFT',
       'Introduction_CRIME_COUNT_4_MONTH_BACKWARD_SHIFT',
       'Méfait_CRIME_COUNT_4_MONTH_BACKWARD_SHIFT',
       'Vol dans / sur véhicule à moteur_CRIME_COUNT_4_MONTH_BACKWARD_SHIFT',
 

In [46]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48552 entries, 4 to 69232
Data columns (total 41 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   temperature_2m_max (°C)_MEAN                                         48552 non-null  float64
 1   temperature_2m_min (°C)_MEAN                                         48552 non-null  float64
 2   temperature_2m_mean (°C)_MEAN                                        48552 non-null  float64
 3   shortwave_radiation_sum (MJ/m²)_MEAN                                 48552 non-null  float64
 4   precipitation_sum (mm)_MEAN                                          48552 non-null  float64
 5   rain_sum (mm)_MEAN                                                   48552 non-null  float64
 6   snowfall_sum (cm)_MEAN                                               48552 non-null  float64
 7   wind

## Normalize 'X_train'

In [47]:
# Create a RobustScaler object
scaler = RobustScaler()

In [49]:
scaler.fit(X_train)
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

## Upsampling feature space (SMOTE)

In [50]:
from imblearn.over_sampling import SMOTE

In [51]:
sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train_normalized, y_train)

In [52]:
print("Resampled Training set shape: ", X_res.shape, y_res.shape)

Resampled Training set shape:  (114270, 41) (114270, 1)


In [53]:
y_res.value_counts()

FIRE_RISK_1_MONTH_FORWARD_SHIFT
4                                  22854
3                                  22854
2                                  22854
1                                  22854
0                                  22854
dtype: int64

# Output file

In [54]:
X_res = pd.DataFrame(X_res)
X_test_normalized = pd.DataFrame(X_test_normalized)
y_res = pd.DataFrame(y_res)

In [55]:
# Output file to folder
file_name = 'df_x_res.csv'
if os.path.exists(file_name):
    os.remove(file_name)
X_res.to_csv('Data/Processed_Datasets/Train_test_tables/' + file_name)


In [56]:
# Output file to folder
file_name = 'df_x_test_normalized.csv'
if os.path.exists(file_name):
    os.remove(file_name)
X_test_normalized.to_csv('Data/Processed_Datasets/Train_test_tables/' + file_name)


In [57]:
# Output file to folder
file_name = 'df_y_res.csv'
if os.path.exists(file_name):
    os.remove(file_name)
y_res.to_csv('Data/Processed_Datasets/Train_test_tables/' + file_name)


In [58]:
# Output file to folder
file_name = 'df_y_test.csv'
if os.path.exists(file_name):
    os.remove(file_name)
y_test.to_csv(
    'Data/Processed_Datasets/Train_test_tables/' + file_name)
